#### Lab : Resampling methods

We will learn the following methods : 
- Cross validation : Primarily used for test error estimation
- Bootstrap : Primarily used for standard deviation estimation

In [ ]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data 
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

We are going to use the `Auto` dataset

In [74]:
data = load_data('Auto')
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,70,1
buick skylark 320,15.0,8,350.0,165,3693,11.5,70,1
plymouth satellite,18.0,8,318.0,150,3436,11.0,70,1
amc rebel sst,16.0,8,304.0,150,3433,12.0,70,1
ford torino,17.0,8,302.0,140,3449,10.5,70,1


### Validation set approach

Here we split the dataset into 2 parts, one where we train and the other where we test. 
Benefits : 
1. Gives us more precise estimates of error

Problems : 
1. Tends to overestimate the error because we are not training the dataset on every sample
2. Depending upon on the testing and training size we have variability in the results

In [9]:
data_train, data_validate = train_test_split(data, test_size=196, random_state=0)
data_train.shape, data_validate.shape

((196, 8), (196, 8))

In [11]:
# Fitting a linear regression model mpg ~ horsepower
hp_mm = MS(["horsepower"])
x_train = hp_mm.fit_transform(data_train)
x_validate  = hp_mm.transform(data_validate)
y_train = data_train.loc[:, "mpg"]
y_validate = data_validate.loc[:, "mpg"]   

linear_model = sm.OLS(y_train, x_train).fit()
preds = linear_model.predict(x_validate)
mse = np.mean((y_validate - preds)**2) 
mse


23.61661706966988

In [46]:
# Defining a function to calculate the MSE of a model

def mse(data, terms, response, test_size=196): 
    mm = MS(terms)
    data_train, data_validate = train_test_split(data, test_size=test_size, random_state=0)
    x_train = mm.fit_transform(data_train)
    x_validate  = mm.transform(data_validate)
    y_train = data_train.loc[:, response]
    y_validate = data_validate.loc[:, response]
    linear_model = sm.OLS(y_train, x_train).fit()
    preds = linear_model.predict(x_validate)
    mse = np.mean((y_validate - preds) ** 2)
    return mse

mse(data, ["horsepower"], "mpg", test_size=0.2)

22.026387306979778

In [51]:
MSE = np.zeros(3)

for i, degree in enumerate(range(1,4)) : 
    terms = poly("horsepower", degree)
    MSE[i] = mse(data, [terms], "mpg", test_size=0.2)
    
MSE

array([22.02638731, 16.01289462, 15.91065108])

So we see that maybe have a quadratic fit of `horsepower` has a sharp decrease to MSE. Though there is no evidence of improvement in cubic term

#### Cross Validation
Simplest way to use cross validation is `sklearn` but `sklearn` has a different interface than `statsmodels` hence we will be using the wrapper in `sklearn_sm()` to use the same

Arguments to `cross_validate()` are : 
1. fit()
2. predict()
3. score()

`KFold()` to partition data into the components we want to test on. 
`KFold(n_splits = n, random_state = 0)`

`ShuffleSplit()` -> makes things more easier

In [57]:
hp_model = sklearn_sm(sm.OLS, MS(["horsepower"]))
x, y = data.drop(columns=["mpg"]), data.loc[:, "mpg"]

# Here we are trying LOOCV
cv_results = cross_validate(hp_model, x,y, cv = data.shape[0])

cv_err = np.mean(cv_results["test_score"])
cv_err

24.23151351792924

In [ ]:
# Complex fitting for polynomial with different degree
cv_error = np.zeros(5)
model = sklearn_sm(sm.OLS)
response = np.array(data["horsepower"])

for i, degree in enumerate(range(1,6)) : 
    # Basically create a set of polynomial features
    x = np.power.outer(response, np.arange(degree+1))
    cv = cross_validate(model, x, y, cv = data.shape[0])
    
    cv_error[i] = np.mean(cv["test_score"])
    
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

We see gains for going from linear to quadractic, we don't see any further gains

In [64]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, random_state=0, shuffle=True)

model = sklearn_sm(sm.OLS)
response = np.array(data["horsepower"])

for i, degree in enumerate(range(1,6)) : 
    # Basically create a set of polynomial features
    x = np.power.outer(response, np.arange(degree+1))
    cv_results = cross_validate(model, x, y, cv = cv)
    
    cv_error[i] = np.mean(cv_results["test_score"])
    
    
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

Still no gains going from linear to quadratic

In [90]:
# Using shuffle split
hp_model = sklearn_sm(sm.OLS, MS(["horsepower"]))
validation = ShuffleSplit(n_splits=10, test_size = 196, random_state=0)

results = cross_validate(hp_model,data.drop(['mpg'], axis=1), data['mpg'],cv=validation)
results["test_score"].mean(), results["test_score"].std()  

(23.802232661034168, 1.421845094109185)